In [1]:
import pandas as pd
import networkx as nx
import os
import numpy as np
import warnings
import numpy as np
import matplotlib.pyplot as plt

from circos import CircosPlot
warnings.filterwarnings('ignore')

%matplotlib inline

# Tables to Networks, Networks to Tables

Networks can be represented in a tabular form in two ways: As an adjacency list with edge attributes stored as columnar values, and as a node list with node attributes stored as columnar values.

Storing the network data as a single massive adjacency table, with node attributes repeated on each row, can get unwieldy, especially if the graph is large, or grows to be so. One way to get around this is to store two files: one with node data and node attributes, and one with edge data and edge attributes. 

The Divvy bike sharing dataset is one such example of a network data set that has been stored as such.

# Loading Node Lists and Adjacency Lists

Let's use the Divvy bike sharing data set as a starting point. The Divvy data set is comprised of the following data:

- Stations and metadata (like a node list with attributes saved)
- Trips and metadata (like an edge list with attributes saved)

The `README.txt` file in the Divvy directory should help orient you around the data.

In [2]:
# This block of code checks to make sure that a particular directory is present.
if "divvy_2013" not in os.listdir('datasets/'):
    print('Unzip the divvy_2013.zip file in the datasets folder.')

In [4]:
stations = pd.read_csv('datasets/divvy_2013/Divvy_Stations_2013.csv', parse_dates=['online date'], index_col='id', encoding='utf-8')
stations

name   latitude  longitude  dpcapacity  \
id                                                                     
5           State St & Harrison St  41.873958 -87.627739          19   
13      Wilton Ave & Diversey Pkwy  41.932500 -87.652681          19   
14             Morgan St & 18th St  41.858086 -87.651073          15   
15            Racine Ave & 18th St  41.858181 -87.656487          15   
16             Wood St & North Ave  41.910329 -87.672516          15   
17           Wood St & Division St  41.903320 -87.672730          15   
19           Loomis St & Taylor St  41.869417 -87.660996          15   
20    Sheffield Ave & Kingsbury St  41.909592 -87.653497          15   
21      Aberdeen St & Jackson Blvd  41.877726 -87.654787          15   
22              May St & Taylor St  41.869482 -87.655486          15   
23             Orleans St & Elm St  41.902924 -87.637715          15   
24        Fairbanks Ct & Grand Ave  41.891860 -87.620620          15   
25       Michigan Ave & Pearson St  41.897660 -87.623510          23   
26        McClurg Ct & Illinois St  41.891020 -87.617300          23   
27         Larrabee St & North Ave  41.910210 -87.643500          19   
28      Larrabee St & Menomonee St  41.914680 -87.643320          15   
29        Noble St & Milwaukee Ave  41.900680 -87.662600          15   
30      Ashland Ave & Augusta Blvd  41.899643 -87.667700          15   
31       Franklin St & Chicago Ave  41.896802 -87.635638          23   
32      Racine Ave & Congress Pkwy  41.874640 -87.657030          19   
33         State St & Van Buren St  41.877181 -87.627844          27   
34       Cannon Dr & Fullerton Ave  41.926756 -87.634429          15   
35       Streeter Dr & Illinois St  41.891071 -87.612200          35   
36      Franklin St & Jackson Blvd  41.877708 -87.635321          27   
37          Dearborn St & Adams St  41.879356 -87.629791          19   
42          Wabash Ave & Cermak Rd  41.853239 -87.625337          15   
43    Michigan Ave & Washington St  41.883893 -87.624649          43   
44          State St & Randolph St  41.884730 -87.627734          27   
45    Michigan Ave & Congress Pkwy  41.876066 -87.624433          15   
46            Wells St & Walton St  41.899930 -87.634430          19   
..                             ...        ...        ...         ...   
322          Kimbark Ave & 53rd St  41.799568 -87.594747          15   
323     Sheridan Rd & Lawrence Ave  41.969517 -87.654691          15   
324   Stockton Dr & Wrightwood Ave  41.931320 -87.638742          15   
325        Clark St & Winnemac Ave  41.973385 -87.668365          15   
326          Clark St & Leland Ave  41.967096 -87.667429          11   
327    Sheffield Ave & Webster Ave  41.921687 -87.653714          19   
328            Ellis Ave & 58th St  41.788746 -87.601334          15   
329  Lake Shore Dr & Diversey Pkwy  41.932684 -87.636250          15   
330       Lincoln Ave & Addison St  41.946176 -87.673308          19   
331      Halsted St & Blackhawk St  41.908540 -87.648568          19   
332     Halsted St & Diversey Pkwy  41.933341 -87.648747          15   
333     Ashland Ave & Blackhawk St  41.907066 -87.667252          15   
334    Lake Shore Dr & Belmont Ave  41.940775 -87.639192          19   
335          Calumet Ave & 35th St  41.831379 -87.618034          15   
336    Cottage Grove Ave & 47th St  41.809855 -87.606755          15   
337         Clark St & Chicago Ave  41.896544 -87.630931          19   
338          Calumet Ave & 18th St  41.857611 -87.619407          15   
339          Emerald Ave & 31st St  41.838198 -87.645143          11   
340      Clark St & Wrightwood Ave  41.929546 -87.643118          15   
341              Adler Planetarium  41.866095 -87.607267          19   
342          Wolcott Ave & Polk St  41.871262 -87.673688          15   
343    Racine Ave & Wrightwood Ave  41.928887 -87.658971          15   
344     Wolcott Ave & Lawrence Ave  41.968641 -87.676335          15  

In [7]:
trips = pd.read_csv('datasets/divvy_2013/Divvy_Trips_2013.csv', 
                    parse_dates=['starttime', 'stoptime'], 
                    index_col=['trip_id'])
trips = trips.sort()
trips

starttime            stoptime  bikeid  tripduration  \
trip_id                                                                 
3940    2013-06-27 01:06:00 2013-06-27 09:46:00     914         31177   
4095    2013-06-27 12:06:00 2013-06-27 12:11:00     480           301   
4113    2013-06-27 11:09:00 2013-06-27 11:11:00     711           140   
4118    2013-06-27 12:11:00 2013-06-27 12:16:00     480           316   
4119    2013-06-27 11:12:00 2013-06-27 11:13:00     711            87   
4134    2013-06-27 11:24:00 2013-06-27 14:38:00     145         11674   
4162    2013-06-27 11:39:00 2013-06-27 16:01:00     711         15758   
4192    2013-06-27 12:15:00 2013-06-27 12:16:00     303            60   
4216    2013-06-27 13:00:00 2013-06-27 13:03:00     907           171   
4255    2013-06-27 13:18:00 2013-06-27 19:34:00     907         22549   
4263    2013-06-27 14:39:00 2013-06-27 14:40:00     145            62   
4275    2013-06-27 14:44:00 2013-06-27 14:45:00      77            64   
4288    2013-06-27 14:56:00 2013-06-27 14:57:00     524            66   
4289    2013-06-27 14:57:00 2013-06-27 15:05:00      78           487   
4291    2013-06-27 14:58:00 2013-06-27 15:05:00      77           433   
4316    2013-06-27 15:06:00 2013-06-27 15:09:00      77           123   
4342    2013-06-27 15:13:00 2013-06-27 15:27:00      77           852   
4343    2013-06-27 15:09:00 2013-06-27 15:14:00     587           272   
4345    2013-06-27 16:14:00 2013-06-27 16:15:00     711            83   
4346    2013-06-27 15:14:00 2013-06-28 00:18:00     145         32646   
4350    2013-06-27 15:15:00 2013-06-27 15:27:00      78           730   
4378    2013-06-27 15:54:00 2013-06-27 23:15:00     524         26479   
4384    2013-06-27 16:42:00 2013-06-27 22:23:00      78         20457   
4389    2013-06-27 16:34:00 2013-06-27 16:36:00      66           127   
4390    2013-06-27 16:41:00 2013-06-27 16:50:00      66           528   
4415    2013-06-27 18:21:00 2013-06-27 18:37:00     384           973   
4427    2013-06-27 18:32:00 2013-06-27 18:37:00     348           308   
4476    2013-06-27 18:38:00 2013-06-27 19:35:00     152          3401   
4480    2013-06-27 19:40:00 2013-06-27 22:28:00      27         10105   
4490    2013-06-27 18:45:00 2013-06-27 19:03:00     418          1094   
...                     ...                 ...     ...           ...   
1109177 2013-12-31 19:34:00 2013-12-31 20:01:00     825          1632   
1109198 2013-12-31 19:42:00 2013-12-31 19:51:00    2181           524   
1109199 2013-12-31 19:42:00 2013-12-31 19:51:00    1978           523   
1109201 2013-12-31 19:44:00 2013-12-31 20:01:00    1304          1003   
1109202 2013-12-31 19:55:00 2013-12-31 20:07:00     188           712   
1109203 2013-12-31 20:02:00 2013-12-31 20:11:00     171           548   
1109222 2013-12-31 20:12:00 2013-12-31 20:25:00    2484           804   
1109223 2013-12-31 20:30:00 2013-12-31 20:37:00     435           390   
1109224 2013-12-31 20:31:00 2013-12-31 20:38:00    2676           387   
1109231 2013-12-31 20:38:00 2014-01-01 12:08:00    2676         55776   
1109233 2013-12-31 20:43:00 2013-12-31 20:51:00    1400           470   
1109240 2013-12-31 20:51:00 2013-12-31 21:05:00      36           819   
1109256 2013-12-31 21:07:00 2013-12-31 21:10:00    1917           209   
1109257 2013-12-31 21:12:00 2013-12-31 21:16:00    1917           284   
1109275 2013-12-31 21:36:00 2013-12-31 22:00:00    2799          1421   
1109276 2013-12-31 21:36:00 2013-12-31 22:01:00    2859          1493   
1109277 2013-12-31 21:36:00 2013-12-31 22:01:00    2642          1447   
1109278 2013-12-31 21:38:00 2013-12-31 22:00:00    2316          1309   
1109279 2013-12-31 21:50:00 2013-12-31 22:07:00    1667           987   
1109280 2013-12-31 21:55:00 2013-12-31 22:04:00     171           573   
1109283 2013-12-31 22:03:00 2013-12-31 22:13:00     198           650   
1109308 2013-12-31 22:10:00 2013-12-31 22:16:00    2931           353   
110

At this point, we have our `stations` and `trips` data loaded into memory. 

How we construct the graph depends on the kind of questions we want to answer, which makes the definition of the "unit of consideration" (or the entities for which we are trying to model their relationships) is extremely important. 

Let's try to answer the question: "What are the most popular trip paths?" In this case, the bike station is a reasonable "unit of consideration", so we will use the bike stations as the nodes. 

To start, let's initialize an directed graph `G`.

In [8]:
G = nx.DiGraph()

Then, let's iterate over the `stations` DataFrame, and add in the node attributes.

In [9]:
for r, d in stations.iterrows(): # call the pandas DataFrame row-by-row iterator
    G.add_node(r,   attr_dict=d.to_dict()) 

In order to answer the question of "which stations are important", we need to specify things a bit more. Perhaps a measure such as **betweenness centrality** or **degree centrality** may be appropriate here.

The naive way would be to iterate over all the rows. Go ahead and try it at your own risk - it may take a long time :-). Alternatively, I would suggest doing a `pandas` `groupby`.

In [10]:
# # Run the following code at your own risk :)
# for r, d in trips.iterrows():
#     start = d['from_station_id']
#     end = d['to_station_id']
#     if (start, end) not in G.edges():
#         G.add_edge(start, end, count=1)
#     else:
#         G.edge[start][end]['count'] += 1

In [11]:
for (start, stop), d in trips.groupby(['from_station_id', 'to_station_id']):
    G.add_edge(start, stop, count=len(d))

In [12]:
G.edges(data=True)

[(5, 5, {'count': 232}),
 (5, 13, {'count': 1}),
 (5, 14, {'count': 15}),
 (5, 15, {'count': 9}),
 (5, 16, {'count': 4}),
 (5, 17, {'count': 7}),
 (5, 19, {'count': 13}),
 (5, 21, {'count': 16}),
 (5, 22, {'count': 18}),
 (5, 23, {'count': 5}),
 (5, 24, {'count': 36}),
 (5, 25, {'count': 62}),
 (5, 26, {'count': 55}),
 (5, 27, {'count': 4}),
 (5, 28, {'count': 4}),
 (5, 29, {'count': 12}),
 (5, 31, {'count': 8}),
 (5, 32, {'count': 14}),
 (5, 33, {'count': 58}),
 (5, 34, {'count': 7}),
 (5, 35, {'count': 83}),
 (5, 36, {'count': 238}),
 (5, 37, {'count': 132}),
 (5, 42, {'count': 47}),
 (5, 43, {'count': 55}),
 (5, 44, {'count': 71}),
 (5, 45, {'count': 43}),
 (5, 46, {'count': 3}),
 (5, 47, {'count': 65}),
 (5, 48, {'count': 3}),
 (5, 49, {'count': 141}),
 (5, 50, {'count': 24}),
 (5, 51, {'count': 69}),
 (5, 52, {'count': 86}),
 (5, 53, {'count': 25}),
 (5, 54, {'count': 13}),
 (5, 55, {'count': 22}),
 (5, 56, {'count': 18}),
 (5, 57, {'count': 49}),
 (5, 58, {'count': 2}),
 (5, 59, 

In [13]:
len(G.edges())

44422

In [15]:
len(G.nodes())

300

### Exercise

Flex your memory muscles: can you make a scatter plot of the distribution of the number edges that have a certain number of trips? 

The key should be the number of trips between two nodes, and the value should be the number of edges that have that number of trips.

In [ ]:
from collections import Counter
# Count the number of edges that have x trips recorded on them.
trip_count_distr = ______________________________

# Then plot the distribution of these
plt.scatter(_______________, _______________, alpha=0.1)
plt.yscale('log')
plt.xlabel('num. of trips')
plt.ylabel('num. of edges')

### Exercise

Create a new graph, and filter out the edges such that only those with more than 100 trips taken (i.e. `count >= 100`) are left.

In [ ]:
# Filter the edges to just those with more than 100 trips.
G_filtered = G.copy()
for u, v, d in G.edges(data=True):
    # Fill in your code here.
        
len(G_filtered.edges())

Let's now try drawing the graph.

### Exercise

Use `nx.draw(my_graph)` to draw the filtered graph to screen.

In [ ]:
# Fill in your code here.


### Exercise

Try visualizing the graph using a CircosPlot. Order the nodes by their connectivity in the **original** graph, but plot only the **filtered** graph edges.

In [ ]:
nodes = sorted(_________________, key=lambda x:_________________)
edges = ___________
edgeprops = dict(alpha=0.1)
nodecolor = plt.cm.viridis(np.arange(len(nodes)) / len(nodes)) 
fig = plt.figure(figsize=(6,6))
ax = fig.add_subplot(111)
c = CircosPlot(nodes, edges, radius=10, ax=ax, fig=fig, edgeprops=edgeprops, nodecolor=nodecolor)
c.draw()
plt.savefig('images/divvy.png', dpi=300)

In this visual, nodes are sorted from highest connectivity to lowest connectivity in the **unfiltered** graph.

Edges represent only trips that were taken >100 times between those two nodes.

Some things should be quite evident here. There are lots of trips between the highly connected nodes and other nodes, but there are local "high traffic" connections between stations of low connectivity as well (nodes in the top-right quadrant).

# Saving NetworkX Graph Files

NetworkX's API offers many formats for storing graphs to disk. If you intend to work exclusively with NetworkX, then pickling the file to disk is probably the easiest way.

To write to disk: 

    nx.write_gpickle(G, handle)

To load from disk:
    
    G = nx.read_gpickle(handle)

In [ ]:
nx.write_gpickle(G, 'datasets/divvy_2013/divvy_graph.pkl')

In [ ]:
G = nx.read_gpickle('datasets/divvy_2013/divvy_graph.pkl')